In [15]:
!pip install Faker
!pip install psycopg2

In [16]:
import random
from faker import Faker
from datetime import datetime, timedelta
import uuid

In [17]:
def generate_sample_data():
    fake = Faker("es_ES")

    # Generate 50 fake users
    users = []
    for i in range(0, 50):
        user_id = fake.uuid4()
        users.append(
            {
                "user_id": user_id,
                "nombre": fake.name(),
                "email": fake.email(),
                "telefono": fake.phone_number(),
                "direccion": fake.address(),
                "fecha_registro": (
                    datetime.now() - timedelta(days=random.randint(180, 365))
                ).strftime("%Y-%m-%d %H:%M:%S"),
            }
        )
    print("--- 50 Sample Users (Salvadoran Spanish) ---")
    print(users[:10])

    # Generate 6 months of billing data for each user
    billing_data = []
    service = "Internet Residencial"
    base_amount = 35.00
    today = datetime.now()

    for user in users:
        for month_offset in range(6, 0, -1):
            bill_date = today - timedelta(days=30 * month_offset)
            amount = base_amount + round(random.uniform(-5.00, 5.00), 2)
            status = random.choice(["Cancelado", "Pendiente"])

            billing_id = fake.uuid4()
            billing_data.append(
                {
                    "billing_id": billing_id,
                    "user_id": user["user_id"],
                    "fecha_factura": bill_date.strftime("%Y-%m-%d 00:00:00"),
                    "monto": amount,
                    "estado": status,
                    "servicio": service,
                }
            )
    print("\n--- 6 Months of Billing Data for 'Internet Residencial' ---")
    print(billing_data[:10])
    return users, billing_data

In [18]:
users, billing_data = generate_sample_data()

--- 50 Sample Users (Salvadoran Spanish) ---
[{'user_id': '8184dc78-b2ff-4b57-b259-6d7afa2c98cb', 'nombre': 'Victoria Atienza Cepeda', 'email': 'qquintanilla@example.org', 'telefono': '+34 985872006', 'direccion': 'Avenida de Isidro Quirós 14\nGranada, 15544', 'fecha_registro': '2025-05-03 20:46:16'}, {'user_id': 'c315f112-b3fb-4781-8e61-4409ee891946', 'nombre': 'Jacinta Xiomara Alemany Barrios', 'email': 'camilarincon@example.org', 'telefono': '+34941 207 479', 'direccion': 'Callejón de Severiano Tapia 17 Apt. 22 \nMelilla, 28256', 'fecha_registro': '2024-12-11 20:46:16'}, {'user_id': 'dda7526a-14a8-46a3-b80e-617dd6d7e0d8', 'nombre': 'Segismundo Hurtado', 'email': 'bpou@example.net', 'telefono': '+34 921 61 09 90', 'direccion': 'Urbanización Ana Belén Soriano 6\nGirona, 25421', 'fecha_registro': '2025-01-07 20:46:16'}, {'user_id': 'd7aa5272-447e-478b-a2e7-de3439f36ba0', 'nombre': 'Mamen Galán Rueda', 'email': 'petronaquiroga@example.com', 'telefono': '+34923 884 784', 'direccion': 'Cu

In [19]:
# Parámetros de conexión MYSQL
config = {
    "user": "rasa_user",
    "password": "icc115",
    "host": "127.0.0.1",
    "database": "clientes_db",
    'port': '5432'
}

In [20]:
import psycopg2
from psycopg2 import DatabaseError

def insert_multiple_records(records, sql_query):
    """Inserta múltiples registros en PostgreSQL utilizando psycopg2."""
    conn = None # Inicializar conn fuera del try para el bloque finally
    try:
        # Establecer la conexión con la base de datos
        conn = psycopg2.connect(**config)
        cursor = conn.cursor()

        # Ejecutar la inserción masiva
        # Note: psycopg2 usa '%s' como marcador de posición, no '%s' ni '?'
        cursor.executemany(sql_query, records)

        # Confirmar los cambios (obligatorio para guardar los datos)
        conn.commit()

        # psycopg2 no tiene rowcount tan fácilmente después de executemany,
        # pero podemos usar el del cursor.
        print(f"{cursor.rowcount} registros insertados correctamente.")

    except DatabaseError as e:
        print(f"Error de base de datos al insertar múltiples registros: {e}")
        if conn:
            conn.rollback() # Revertir cambios ante un error
        # Re-lanzar la excepción para que el código que llama sepa que falló
        # O se puede simplemente retornar
        # raise 
        
    except Exception as e:
        # Capturar otros errores como ConfigurationError, etc.
        print(f"Error inesperado: {e}")
        
    finally:
        # Cerrar el cursor y la conexión
        if conn:
            if cursor:
                cursor.close()
            conn.close()
            print("Conexión a PostgreSQL cerrada.")


def convertir_dict_a_tupla(lista_de_diccionarios, orden_columnas):
    """
    Convierte una lista de diccionarios a una lista de tuplas,
    garantizando el orden de los valores. (Esta función NO requiere cambios de driver)

    Args:
        lista_de_diccionarios (list): La lista de diccionarios a convertir.
        orden_columnas (list): Una lista de strings con las claves en el orden deseado.

    Returns:
        list: Una lista de tuplas con los valores ordenados.
    """
    records_as_tuples = []
    for record in lista_de_diccionarios:
        try:
            # Obtener los valores en el orden especificado
            tuple_record = tuple(record[key] for key in orden_columnas)
            records_as_tuples.append(tuple_record)
        except KeyError as e:
            print(
                f"Advertencia: El diccionario {record} no contiene la clave {e}. Se omite."
            )
    return records_as_tuples

In [21]:
# Insertando usuarios
insert_users_sql_query = "INSERT INTO users (user_id, nombre, email, telefono, direccion, fecha_registro) VALUES (%s, %s, %s, %s, %s, %s)"
tuplas_users = convertir_dict_a_tupla(
    users, ["user_id", "nombre", "email", "telefono", "direccion", "fecha_registro"]
)
print(tuplas_users[:10])
insert_multiple_records(tuplas_users, insert_users_sql_query)

[('8184dc78-b2ff-4b57-b259-6d7afa2c98cb', 'Victoria Atienza Cepeda', 'qquintanilla@example.org', '+34 985872006', 'Avenida de Isidro Quirós 14\nGranada, 15544', '2025-05-03 20:46:16'), ('c315f112-b3fb-4781-8e61-4409ee891946', 'Jacinta Xiomara Alemany Barrios', 'camilarincon@example.org', '+34941 207 479', 'Callejón de Severiano Tapia 17 Apt. 22 \nMelilla, 28256', '2024-12-11 20:46:16'), ('dda7526a-14a8-46a3-b80e-617dd6d7e0d8', 'Segismundo Hurtado', 'bpou@example.net', '+34 921 61 09 90', 'Urbanización Ana Belén Soriano 6\nGirona, 25421', '2025-01-07 20:46:16'), ('d7aa5272-447e-478b-a2e7-de3439f36ba0', 'Mamen Galán Rueda', 'petronaquiroga@example.com', '+34923 884 784', 'Cuesta de Eusebia Rivera 10 Piso 1 \nLa Rioja, 40462', '2025-03-22 20:46:16'), ('451242f7-a9e0-410e-80ab-76b7a140fe80', 'Adrián del Sevillano', 'escobarsabina@example.org', '+34 924 940 849', 'Urbanización Aurelio Espada 943 Puerta 3 \nSevilla, 01166', '2025-02-17 20:46:16'), ('a9c1cfdf-d653-436b-9974-dc07070fa101', 'Aa

In [22]:
# Insertando facturas
insert_billing_data_sql_query = "INSERT INTO billing_data (billing_id, user_id, fecha_factura, monto, estado, producto_id) VALUES (%s, %s, %s, %s, %s, %s)"

billing_data_updated = []
productos = [
    "c9a0b3e1-ee0c-44d0-accb-959b75cec5ad",
    "d5c151fd-0c02-49c3-858a-ccd80a870a76",
    "d5c151fd-0c02-49c3-858a-ccd80a870a76",
]
for record in billing_data:
    record["producto_id"] = productos[random.randint(0, 2)]
    billing_data_updated.append(record)
tuplas_billing_data = convertir_dict_a_tupla(
    billing_data,
    ["billing_id", "user_id", "fecha_factura", "monto", "estado", "producto_id"],
)
print(tuplas_billing_data[:10])
insert_multiple_records(tuplas_billing_data, insert_billing_data_sql_query)

[('dbfe3afd-5ca8-40ed-9ddc-c49f470c0b4d', '8184dc78-b2ff-4b57-b259-6d7afa2c98cb', '2025-05-21 00:00:00', 31.94, 'Pendiente', 'c9a0b3e1-ee0c-44d0-accb-959b75cec5ad'), ('a8accfec-b917-4cf3-9445-995acf27d128', '8184dc78-b2ff-4b57-b259-6d7afa2c98cb', '2025-06-20 00:00:00', 33.6, 'Cancelado', 'd5c151fd-0c02-49c3-858a-ccd80a870a76'), ('db8c159f-9923-4ff7-9513-f508bd41a52a', '8184dc78-b2ff-4b57-b259-6d7afa2c98cb', '2025-07-20 00:00:00', 34.62, 'Cancelado', 'c9a0b3e1-ee0c-44d0-accb-959b75cec5ad'), ('e1a21d31-6e8b-4184-92b2-88a52544cd00', '8184dc78-b2ff-4b57-b259-6d7afa2c98cb', '2025-08-19 00:00:00', 38.45, 'Cancelado', 'c9a0b3e1-ee0c-44d0-accb-959b75cec5ad'), ('5c17122b-3833-48a2-9e97-bdc80e36b904', '8184dc78-b2ff-4b57-b259-6d7afa2c98cb', '2025-09-18 00:00:00', 37.41, 'Pendiente', 'd5c151fd-0c02-49c3-858a-ccd80a870a76'), ('e01f0637-2c4a-4085-9d4b-de7f0ab13902', '8184dc78-b2ff-4b57-b259-6d7afa2c98cb', '2025-10-18 00:00:00', 37.4, 'Cancelado', 'd5c151fd-0c02-49c3-858a-ccd80a870a76'), ('362fc97c-

In [23]:
import json


def cargar_json(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            data = json.load(file)
            print("Datos cargados desde el archivo:")
            print(data[:1])
            print("El tipo de dato cargado es:", type(data))
            return data
    except FileNotFoundError:
        print(f"Error: El archivo '{file_path}' no fue encontrado.")
    except json.JSONDecodeError:
        print(
            "Error: No se pudo decodificar el JSON. Asegúrate de que el formato sea correcto."
        )

### Cargando conversaciones

In [24]:
conversations = []
new_client_sample_conversations_file_path = "./new_client_sample_conversations.json"
new_client_sample_conversations = cargar_json(new_client_sample_conversations_file_path)
conversations.extend(new_client_sample_conversations)

billing_sample_conversations_file_path = "./billing_sample_conversations.json"
billing_sample_conversations = cargar_json(billing_sample_conversations_file_path)
conversations.extend(billing_sample_conversations)

conexion_issues_conversations_file_path = "./conexion_issues_conversations.json"
conexion_issues_conversations = cargar_json(conexion_issues_conversations_file_path)
conversations.extend(conexion_issues_conversations)

print(conversations[:2])

Datos cargados desde el archivo:
[{'cliente': {'nombre_completo': 'Laura Patricia Vásquez Orellana', 'telefono': '7987-6543'}, 'satisfaccion': 5, 'historial': [{'sender': 'cliente', 'text': 'Hola, estoy interesada en contratar un plan de internet residencial. ¿Qué opciones tienen?'}, {'sender': 'agente', 'text': '¡Buenos días, Sra. Vásquez! Tenemos varios planes residenciales con diferentes velocidades. Para poderle dar la mejor recomendación, ¿podría indicarme su DUI para verificar la cobertura en su área?'}, {'sender': 'cliente', 'text': 'Sí, claro. Mi DUI es 09876543-2. Vivo en la Colonia Escalón.'}, {'sender': 'agente', 'text': 'Gracias. He verificado y su zona cuenta con cobertura de fibra óptica. Le recomiendo nuestro plan de 100 Mbps, que incluye una instalación gratuita por promoción. ¿Le gustaría que le brinde más detalles?'}, {'sender': 'cliente', 'text': 'Me parece muy bien. Sí, por favor, envíeme la información completa al correo electrónico.'}, {'sender': 'agente', 'text':

In [25]:
import psycopg2
def load_users_from_postgres():
    """
    Carga todos los registros de la tabla 'users' desde PostgreSQL.
    
    Args:
        config (dict): Diccionario de parámetros de conexión a PostgreSQL.
        
    Returns:
        list: Una lista de diccionarios (registros de usuarios) o None en caso de error.
    """
    conn = None
    try:
        # 1. Establecer la conexión con la base de datos PostgreSQL
        conn = psycopg2.connect(**config)
        
        # 2. El cursor por defecto de psycopg2 NO devuelve diccionarios.
        #    Usamos un módulo extra para obtener un cursor que devuelva diccionarios.
        from psycopg2.extras import RealDictCursor 
        cursor = conn.cursor(cursor_factory=RealDictCursor)
        
        # 3. Ejecutar la consulta SQL (es casi idéntica, solo se eliminan las funciones MySQL)
        cursor.execute(
            "SELECT user_id, nombre, email, telefono, direccion, fecha_registro FROM users"
        )
        users_from_db = cursor.fetchall()
        
        print("Usuarios cargados desde PostgreSQL:")
        print(users_from_db[:5])
        return users_from_db
        
    except DatabaseError as e:
        # Manejo de errores específico de psycopg2
        print(f"Error de base de datos al cargar usuarios: {e}")
        return None
        
    except Exception as e:
        # Otros errores (configuración, etc.)
        print(f"Error inesperado al cargar usuarios: {e}")
        return None
        
    finally:
        # Cerrar el cursor y la conexión
        if conn:
            # No es necesario verificar conn.is_connected() con psycopg2 antes de close()
            cursor.close()
            conn.close()
            print("Conexión a PostgreSQL cerrada.")

In [26]:
import random

conversations_data = []
users_db = load_users_from_postgres()
for conversation in conversations:
    # Generar un número entero aleatorio entre 1 y 10 (inclusive)
    random_int = random.randint(0, 49)
    ramdom_user = users_db[random_int]
    session_id = str(uuid.uuid4())
    random_days_ago = random.randint(0, 180)
    random_date = datetime.now() - timedelta(days=random_days_ago)

    start_time = random_date  # Timestamp inicial de la conversación

    conversation_data = {}
    conversation_data["session_id"] = session_id
    conversation_data["user_id"] = ramdom_user["user_id"]
    # Array para almacenar los mensajes de la conversación
    conversation_mensajes = []

    # Iterar sobre los mensajes de la conversación
    for i, message in enumerate(conversation["historial"]):
        random_minutes = random.randint(0, 15)
        current_time = start_time + timedelta(minutes=random_minutes)
        message_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
        message_id = str(uuid.uuid4())

        message_data = {
            "session_id": session_id,
            "message_id": message_id,
            "timestamp": message_time,
            "sender": message["sender"],
            "text": message["text"],
        }

        # Agregar fecha y hora de inicio y fin de la conversación
        if i == 0:
            conversation_data["fecha_inicio"] = message_time
        if i == len(conversation["historial"]) - 1:
            conversation_data["fecha_fin"] = message_time

        # Se agrega el mensaje al array de mensajes de la conversación
        conversation_mensajes.append(message_data)

        start_time = current_time  # Actualizar start_time para el próximo mensaje

    # Se agrega el array de mensajes a la conversación
    conversation_data["messages"] = conversation_mensajes

    # Agregando support tickets
    if "support_ticket" in conversation:
        # Generar un support ticket para la conversación

        ticket_id = str(uuid.uuid4())
        ticket_created_at = conversation_data["fecha_fin"]
        estado = conversation["support_ticket"]["estado"]
        problema = conversation["support_ticket"]["problema"]
        descripcion = conversation["support_ticket"]["descripcion"]
        prioridad = conversation["support_ticket"]["prioridad"]

        support_ticket_random_hours = random.randint(
            0, 24 * 3
        )  # Hasta 3 días después de la creación
        support_ticket_fecha_cierre = start_time + timedelta(
            hours=support_ticket_random_hours
        )

        fecha_cierre = support_ticket_fecha_cierre if estado == "Cerrado" else None

        support_ticket = {
            "ticket_id": ticket_id,
            "user_id": conversation_data["user_id"],
            "prioridad": prioridad,
            "estado": estado,
            "problema": problema,
            "descripcion": descripcion,
            "fecha_creacion": conversation_data["fecha_fin"],
            "fecha_cierre": fecha_cierre,
        }

        conversation_data["support_ticket"] = support_ticket

    # Agregar feedback
    feedback_id = str(uuid.uuid4())
    if "feedback" in conversation:
        # Generar un feedback para la conversación
        rating = conversation["feedback"]["rating"]
        comentarios = conversation["feedback"]["comentarios"]
    else:
        rating = random.randint(1, 5)
        comentarios = random.choice(
            [
                "Excelente atención.",
                "Resolvieron mi problema rápidamente.",
                "El servicio podría mejorar.",
                "No quedé satisfecho con la respuesta.",
                "Muy buen soporte técnico.",
                "La conversación fue clara y útil.",
                "Me gustaría una atención más rápida.",
                "El agente fue muy amable.",
                "No entendí la solución propuesta.",
                "Todo perfecto, gracias.",
            ]
        )

    fecha_feedback = conversation_data["fecha_fin"]

    feedback = {
        "feedback_id": feedback_id,
        "session_id": session_id,
        "user_id": conversation_data["user_id"],
        "rating": rating,
        "comentarios": comentarios,
        "fecha_feedback": fecha_feedback,
    }

    conversation_data["feedback"] = feedback
    conversation_data["satisfaccion"] = (
        conversation["satisfaccion"] if "satisfaccion" in conversation else rating
    )

    # Agregar visita técnica
    if "visita_tecnica" in conversation:
        visita_id = str(uuid.uuid4())
        tecnico = conversation["visita_tecnica"]["tecnico"]
        fecha_visita = conversation["visita_tecnica"]["fecha_visita"]

        visita_tecnica_estado_choices = [
            # "Programada",
            # "En Curso",
            "Completada",
            "Cancelada",
            "Cerrada",
        ]
        visita_tecnica_estado = random.choice(visita_tecnica_estado_choices)

        visita_tecnica = {
            "visit_id": visita_id,
            "ticket_id": ticket_id if "support_ticket" in conversation else None,
            "user_id": conversation_data["user_id"],
            "tecnico": tecnico,
            "fecha_visita": fecha_visita,
            "estado": visita_tecnica_estado,
        }

        conversation_data["visita_tecnica"] = visita_tecnica

    conversations_data.append(conversation_data)

print(conversations_data[10])

Usuarios cargados desde PostgreSQL:
[RealDictRow([('user_id', '8184dc78-b2ff-4b57-b259-6d7afa2c98cb'), ('nombre', 'Victoria Atienza Cepeda'), ('email', 'qquintanilla@example.org'), ('telefono', '+34 985872006'), ('direccion', 'Avenida de Isidro Quirós 14\nGranada, 15544'), ('fecha_registro', datetime.datetime(2025, 5, 3, 20, 46, 16))]), RealDictRow([('user_id', 'c315f112-b3fb-4781-8e61-4409ee891946'), ('nombre', 'Jacinta Xiomara Alemany Barrios'), ('email', 'camilarincon@example.org'), ('telefono', '+34941 207 479'), ('direccion', 'Callejón de Severiano Tapia 17 Apt. 22 \nMelilla, 28256'), ('fecha_registro', datetime.datetime(2024, 12, 11, 20, 46, 16))]), RealDictRow([('user_id', 'dda7526a-14a8-46a3-b80e-617dd6d7e0d8'), ('nombre', 'Segismundo Hurtado'), ('email', 'bpou@example.net'), ('telefono', '+34 921 61 09 90'), ('direccion', 'Urbanización Ana Belén Soriano 6\nGirona, 25421'), ('fecha_registro', datetime.datetime(2025, 1, 7, 20, 46, 16))]), RealDictRow([('user_id', 'd7aa5272-447e-

In [27]:
messages = []
feedbacks = []
technical_visits = []
support_tickets = []
conversation_records = []
for conv in conversations_data:
    messages_tuples = [
        (
            msg["session_id"],
            msg["message_id"],
            msg["timestamp"],
            msg["sender"],
            msg["text"],
        )
        for msg in conv["messages"]
    ]
    messages.extend(messages_tuples)

    
    feedbacks_tuples = [
        (
            conv["feedback"]["feedback_id"],
            conv["feedback"]["session_id"],
            conv["feedback"]["user_id"],
            conv["feedback"]["rating"],
            conv["feedback"]["comentarios"],
            conv["feedback"]["fecha_feedback"],
        )
    ]
    feedbacks.extend(feedbacks_tuples)

    if "visita_tecnica" in conv:
        visita = conv["visita_tecnica"]
        technical_visits.append(
            (
                visita["visit_id"],
                visita["ticket_id"],
                visita["user_id"],
                visita["tecnico"],
                visita["fecha_visita"],
                visita["estado"],
            )
        )
    if "support_ticket" in conv:
        support_tickets.append(
            (
                conv["support_ticket"]["ticket_id"],
                conv["support_ticket"]["user_id"],
                conv["support_ticket"]["prioridad"],
                conv["support_ticket"]["estado"],
                conv["support_ticket"]["problema"],
                conv["support_ticket"]["descripcion"],
                conv["support_ticket"]["fecha_creacion"],
                conv["support_ticket"]["fecha_cierre"],
            )
        )
    conversation_records.append(
        (
            conv["session_id"],
            conv["user_id"],
            conv["fecha_inicio"],
            conv["fecha_fin"],
            conv["satisfaccion"],
        )
    )

In [28]:
# Insertando facturas
conversations_data_sql_query = "INSERT INTO conversations (session_id, user_id, fecha_inicio, fecha_fin, satisfaccion) VALUES (%s, %s, %s, %s, %s)"
print(conversation_records[:10])
insert_multiple_records(conversation_records, conversations_data_sql_query)

[('57f33480-a07b-4382-99bf-5c48a2a9195c', '95d52f83-e43e-4d6d-b90d-e027e6ea1282', '2025-08-05 20:46:17', '2025-08-05 21:30:17', 5), ('e06d50d3-b7f5-4c4a-80ce-3b285eb49b10', 'dda7526a-14a8-46a3-b80e-617dd6d7e0d8', '2025-11-08 20:58:17', '2025-11-08 21:27:17', 2), ('4db650cd-1358-4676-8e53-b04b01926b70', '9e920639-28ec-480f-b029-e5c963aa8d6f', '2025-07-13 20:58:17', '2025-07-13 21:42:17', 4), ('be6b2a6d-1832-416f-9b6a-b5a0a5e560da', 'ad7a59a7-684b-46b3-b9ff-a966065b4773', '2025-08-30 20:57:17', '2025-08-30 21:28:17', 1), ('880b2235-6c07-4266-a952-77b07644d69b', 'c00ff91e-af32-4fb5-b8c2-f8041686bd83', '2025-09-10 20:54:17', '2025-09-10 21:22:17', 5), ('db21a3e1-bb4c-4590-a83f-3719e457475d', 'acaabd72-be0d-4ce7-8aa6-6c614f32b193', '2025-08-08 21:00:17', '2025-08-08 21:32:17', 5), ('127b67fa-668c-4626-958a-df81e2460bb7', 'ad7a59a7-684b-46b3-b9ff-a966065b4773', '2025-05-28 20:46:17', '2025-05-28 21:38:17', 2), ('7d93856f-e9a5-4dbe-834e-5ae8d887a0a7', '9f146f39-5128-465b-a177-9a469ab53dbc', '

In [29]:
messages_sql_query = "INSERT INTO messages (session_id, message_id, timestamp, sender, text) VALUES (%s, %s, %s, %s, %s)"
insert_multiple_records(messages, messages_sql_query)

84 registros insertados correctamente.
Conexión a PostgreSQL cerrada.


In [30]:
feedbacks_sql_query = "INSERT INTO feedback (feedback_id, session_id, user_id, rating, comentarios, fecha_feedback) VALUES (%s, %s, %s, %s, %s, %s)"
insert_multiple_records(feedbacks, feedbacks_sql_query)

15 registros insertados correctamente.
Conexión a PostgreSQL cerrada.


In [31]:
support_tickets_sql_query = "INSERT INTO support_tickets (ticket_id, user_id, prioridad, estado, problema, descripcion, fecha_creacion, fecha_cierre) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
insert_multiple_records(support_tickets, support_tickets_sql_query)

7 registros insertados correctamente.
Conexión a PostgreSQL cerrada.


In [32]:
technical_visits_sql_query = "INSERT INTO technical_visits (visit_id, ticket_id, user_id, tecnico, fecha_visita, estado) VALUES (%s, %s, %s, %s, %s, %s)"
insert_multiple_records(technical_visits, technical_visits_sql_query)
print(technical_visits[:1])

3 registros insertados correctamente.
Conexión a PostgreSQL cerrada.
[('12d79280-6c90-4263-a0e3-e248e57f0fa6', 'fa249034-3e2f-4dd0-8fa6-1d4f7ea06966', '451242f7-a9e0-410e-80ab-76b7a140fe80', 'Carlos Pineda', '2025-10-08', 'Cancelada')]
